# データの準備

In [1]:
import os
import keras
import numpy as np
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

os.makedirs("./data", exist_ok = True)

np.savez('./data/train', image=x_train, label=y_train)
np.savez('./data/test', image=x_test, label=y_test)

Using TensorFlow backend.


# ローカルでの動作確認
## ローカル用のDockerコンテナのビルド
tf2.0をベースにしたSageMakerコンパチなDockerfile。

In [2]:
!cat container/Dockerfile

FROM tensorflow/tensorflow:2.0.0a0

RUN pip install sagemaker-containers

# Copies the training code inside the container
COPY tf_codes /opt/ml/code

# Defines train.py as script entry point
ENV SAGEMAKER_PROGRAM train.py
ENV SAGEMAKER_PROGRAM serve.py

ローカルモードでの検証のために、ノートブックインスタンス上にビルド。

In [7]:
!docker build -t tf-2.0 container/.

Sending build context to Docker daemon  33.28kB
Step 1/5 : FROM tensorflow/tensorflow:2.0.0a0
 ---> 2ebc856b5e27
Step 2/5 : RUN pip install sagemaker-containers
 ---> Using cache
 ---> dbe5fb9d9dd5
Step 3/5 : COPY tf_codes /opt/ml/code
 ---> Using cache
 ---> afd31f563dd0
Step 4/5 : ENV SAGEMAKER_PROGRAM train.py
 ---> Using cache
 ---> 8a5cdf3330b5
Step 5/5 : ENV SAGEMAKER_SERVING_MODULE serve.py
 ---> Running in a9799e30a9db
Removing intermediate container a9799e30a9db
 ---> e5475ce1fcbd
Successfully built e5475ce1fcbd
Successfully tagged tf-2.0:latest


In [8]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator

sagemaker_session = sagemaker.Session()

role = get_execution_role()

estimator = Estimator(image_name='tf-2.0',
                      role=role,
                      hyperparameters={'batch_size': 64,'epochs': 1},
                      train_instance_count=1,
                      train_instance_type='local')

In [9]:
estimator.fit({'train': 'file://data'})

Creating tmpchnnvji__algo-1-y1mj3_1 ... 
Attaching to tmpchnnvji__algo-1-y1mj3_12mdone
algo-1-y1mj3_1  | 2019-10-03 07:26:19,950 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-y1mj3_1  | 2019-10-03 07:26:19,968 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-y1mj3_1  | 2019-10-03 07:26:19,984 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-y1mj3_1  | 2019-10-03 07:26:19,998 sagemaker-containers INFO     Invoking user script
algo-1-y1mj3_1  | 
algo-1-y1mj3_1  | Training Env:
algo-1-y1mj3_1  | 
algo-1-y1mj3_1  | {
algo-1-y1mj3_1  |     "module_dir": "/opt/ml/code", 
algo-1-y1mj3_1  |     "channel_input_dirs": {
algo-1-y1mj3_1  |         "train": "/opt/ml/input/data/train"
algo-1-y1mj3_1  |     }, 
algo-1-y1mj3_1  |     "resource_config": {
algo-1-y1mj3_1  |         "hosts": [
algo-1-y1mj3_1  |             "algo-1-y1mj3"
algo-1-y1mj3_1  |         ], 
algo-1-y1mj3_1  |         "curr

In [10]:
predictor = estimator.deploy(instance_type='ml.m4.xlarge', initial_instance_count=1)

W1003 07:26:25.274262 140136904808256 connectionpool.py:662] Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f73c8801b38>: Failed to establish a new connection: [Errno 111] Connection refused',)': /ping
W1003 07:26:25.280159 140136904808256 connectionpool.py:662] Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f73c8801550>: Failed to establish a new connection: [Errno 111] Connection refused',)': /ping
W1003 07:26:25.281096 140136904808256 connectionpool.py:662] Retrying (Retry(total=0, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f73c88010b8>: Failed to establish a new connection: [Errno 111] Connection refused',)': /ping


Attaching to tmptrgmpeet_algo-1-wwa9v_1
algo-1-wwa9v_1  | Traceback (most recent call last):
algo-1-wwa9v_1  |   File "/usr/local/bin/serve", line 10, in <module>
algo-1-wwa9v_1  |     sys.exit(main())
algo-1-wwa9v_1  |   File "/usr/local/lib/python2.7/dist-packages/sagemaker_containers/cli/serve.py", line 19, in main
algo-1-wwa9v_1  |     server.start(env.ServingEnv().framework_module)
algo-1-wwa9v_1  |   File "/usr/local/lib/python2.7/dist-packages/sagemaker_containers/_server.py", line 82, in start
algo-1-wwa9v_1  |     nginx = subprocess.Popen(["nginx", "-c", nginx_config_file])
algo-1-wwa9v_1  |   File "/usr/lib/python2.7/subprocess.py", line 711, in __init__
algo-1-wwa9v_1  |     errread, errwrite)
algo-1-wwa9v_1  |   File "/usr/lib/python2.7/subprocess.py", line 1343, in _execute_child
algo-1-wwa9v_1  |     raise child_exception
algo-1-wwa9v_1  | OSError: [Errno 2] No such file or directory
tmptrgmpeet_algo-1-wwa9v_1 exited with code 1
Aborting on container exit...


Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sagemaker/local/image.py", line 596, in run
    _stream_output(self.process)
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sagemaker/local/image.py", line 657, in _stream_output
    raise RuntimeError("Process exited with code: %s" % exit_code)
RuntimeError: Process exited with code: 1

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sagemaker/local/image.py", line 601, in run
    raise RuntimeError(msg)
RuntimeError: Failed to run: ['docker-compose', '-f', '/tmp/tmptrgmpeet/docker-compose.yaml', 'up', '--build', '--abort-on-container-exit'], Proces

KeyboardInterrupt: 

# コンテナのビルドとECRへの登録

In [12]:
!bash container/build_and_push.sh

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Sending build context to Docker daemon  33.28kB
Step 1/5 : FROM tensorflow/tensorflow:2.0.0a0
 ---> 2ebc856b5e27
Step 2/5 : RUN pip install sagemaker-containers
 ---> Using cache
 ---> dbe5fb9d9dd5
Step 3/5 : COPY tf_codes /opt/ml/code
 ---> Using cache
 ---> afd31f563dd0
Step 4/5 : ENV SAGEMAKER_PROGRAM train.py
 ---> Using cache
 ---> 8a5cdf3330b5
Step 5/5 : ENV SAGEMAKER_SERVING_MODULE serve.py
 ---> Using cache
 ---> e5475ce1fcbd
Successfully built e5475ce1fcbd
Successfully tagged sagemaker-tf2.0-example:latest
The push refers to repository [815969174475.dkr.ecr.us-east-1.amazonaws.com/sagemaker-tf2.0-example]

0f8a9227: Preparing 
b1817ceb: Preparing 
409a88ea: Preparing 

コンテナイメージのパスの取得。

In [13]:
import boto3

client = boto3.client('sts')
account = client.get_caller_identity()['Account']

my_session = boto3.session.Session()
region = my_session.region_name

algorithm_name = 'sagemaker-tf2.0-example'

ecr_image = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(account, region, algorithm_name)

print(ecr_image)

815969174475.dkr.ecr.us-east-1.amazonaws.com/sagemaker-tf2.0-example:latest


## データのS3へのアップロード

In [14]:
bucket_name = sagemaker_session.default_bucket()
input_data = sagemaker_session.upload_data(path='./data', bucket=bucket_name, key_prefix='dataset/mnist')
print('Training data is uploaded to: {}'.format(input_data))

Training data is uploaded to: s3://sagemaker-us-east-1-815969174475/dataset/mnist


In [15]:
estimator = Estimator(image_name=ecr_image,
                      role=role,
                      base_job_name='tf20-example',
                      hyperparameters={'batch_size': 64,'epochs': 1},
                      train_instance_count=1,
                      train_instance_type='ml.p2.xlarge')

In [17]:
estimator.fit({'train': '{}'.format(input_data)})

2019-10-03 07:32:57 Starting - Starting the training job...
2019-10-03 07:32:59 Starting - Launching requested ML instances......
2019-10-03 07:34:00 Starting - Preparing the instances for training...
2019-10-03 07:34:55 Downloading - Downloading input data...
2019-10-03 07:35:12 Training - Downloading the training image.....2019-10-03 07:36:05,425 sagemaker-containers INFO     Invoking user script

Training Env:

{
    "module_dir": "/opt/ml/code", 
    "channel_input_dirs": {
        "train": "/opt/ml/input/data/train"
    }, 
    "resource_config": {
        "hosts": [
            "algo-1"
        ], 
        "network_interface_name": "eth0", 
        "current_host": "algo-1"
    }, 
    "num_cpus": 4, 
    "log_level": 20, 
    "output_intermediate_dir": "/opt/ml/output/intermediate", 
    "input_config_dir": "/opt/ml/input/config", 
    "additional_framework_parameters": {}, 
    "output_data_dir": "/opt/ml/output/data", 
    "output_dir": "/opt/ml/output", 
    "model_dir": "/opt

In [ ]:
predictor = estimator.deploy(instance_type='ml.m4.xlarge', initial_instance_count=1)

In [ ]:
%matplotlib inline
import random
import matplotlib.pyplot as plt

num_samples = 5
indices = random.sample(range(x_test.shape[0] - 1), num_samples)
images, labels = x_test[indices]/255, y_test[indices]

for i in range(num_samples):
    plt.subplot(1,num_samples,i+1)
    plt.imshow(images[i].reshape(28, 28), cmap='gray')
    plt.title(labels[i])
    plt.axis('off')
    
prediction = predictor.predict(images.reshape(num_samples, 28, 28, 1))['predictions']
prediction = np.array(prediction)
predicted_label = prediction.argmax(axis=1)
print('The predicted labels are: {}'.format(predicted_label))

In [ ]:
predictor.delete_endpoint()